In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop


2023-02-18 00:34:28.399800: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 00:34:29.011939: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-18 00:34:29.011997: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-18 00:34:30.506895: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:

# Define a função de perda WGAN
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

# Define o gerador
def build_generator(latent_dim):
    inputs = Input(shape=(latent_dim,))
    x = Dense(7*7*256, use_bias=False)(inputs)
    x = Reshape((7, 7, 256))(x)
    x = Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)
    generator = Model(inputs, x)
    return generator



# Define o discriminador
def build_discriminator():
    inputs = Input(shape=(28, 28, 1))
    x = Conv2D(64, (5, 5), strides=(2, 2), padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, (5, 5), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    x = Dense(1)(x)
    discriminator = Model(inputs, x)
    return discriminator


In [3]:


# Define o treinamento da WGAN
def train_wgan(generator, discriminator, combined, x_train, latent_dim, epochs, batch_size):
    d_loss = []
    g_loss = []
    for epoch in range(epochs):
        for batch in range(x_train.shape[0] // batch_size):
            # Treina o discriminador
            discriminator.trainable = True
            for _ in range(5):
                noise = np.random.normal(0, 1, (batch_size, latent_dim))
                fake_images = generator.predict(noise)
                real_images = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
                d_loss_real = discriminator.train_on_batch(real_images, np.ones(batch_size))
                d_loss_fake = discriminator.train_on_batch(fake_images, -np.ones(batch_size))
                d_loss_batch = 0.5 * np.add(d_loss_real, d_loss_fake)
                for layer in discriminator.layers:
                    weights = layer.get_weights()
                    weights = [np.clip(w, -0.01, 0.01) for w in weights]
                    layer.set_weights(weights)
            # Treina o gerador
            discriminator.trainable = False
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss_batch = combined.train_on_batch(noise, np.ones(batch_size))
        d_loss.append(d_loss_batch)
        g_loss.append(g_loss_batch)
        print('Epoch %d: [Discriminator loss: %f, Generator loss: %f]' % (epoch+1, d_loss_batch, g_loss_batch))
    if (epoch+1) % 10 == 0:
        plot_images(generator)

    return d_loss, g_loss


In [4]:
def plot_images(generator, n_images=10):
    noise = np.random.normal(0, 1, (n_images, 100))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    fig, axs = plt.subplots(1, n_images, figsize=(10, 10))
    cnt = 0
    for i in range(n_images):
        axs[i].imshow(generated_images[cnt, :, :, 0], cmap='gray')
        axs[i].axis('off')
        cnt += 1
        plt.show()

In [5]:
(x_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_train = (x_train - 127.5) / 127.5 # normaliza as imagens para o intervalo [-1, 1]
batch_size = 128
latent_dim = 100

In [6]:
generator = build_generator(latent_dim)
discriminator = build_discriminator()
discriminator.compile(optimizer=RMSprop(lr=0.00005), loss=wasserstein_loss)
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
combined = Model(gan_input, gan_output)
combined.compile(optimizer=RMSprop(lr=0.00005), loss=wasserstein_loss)

2023-02-18 00:34:32.942240: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-18 00:34:32.942258: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-18 00:34:32.942274: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sylens): /proc/driver/nvidia/version does not exist
2023-02-18 00:34:32.942961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
d_loss, g_loss = train_wgan(generator, discriminator, combined, x_train, latent_dim, epochs=50, batch_size=batch_size)

4/4 [==============================] - 0s 23ms/step


4/4 [==============================] - 0s 29ms/step


4/4 [==============================] - 0s 24ms/step


4/4 [==============================] - 0s 28ms/step


4/4 [==============================] - 0s 26ms/step


...

4/4 [==============================] - 0s 29ms/step


4/4 [==============================] - 0s 29ms/step


4/4 [==============================] - 0s 25ms/step
Epoch 10: [Discriminator loss: 0.001265, Generator loss: 0.000163]
4/4 [==============================] - 0s 26ms/step


4/4 [==============================] - 0s 31ms/step


1/4 [======>.......................] - ETA: 0s